# Lecture 8 - PyTorch

This will be the final lecture, today we will first have a brief introduction of deep learning, then we will look at some basics of using PyTorch to implement some simple models in deep learning.

1. Homework due today

# Basic Section (Start)
What is Neural Network:
1. [Recommended youtube video with great visual helpers](https://www.youtube.com/watch?v=aircAruvnKk&list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi&index=4)
2.  Slides on course site
# Basic Section (End)

## Deep Learning Libraries

There are many deep learning libraries available, the most common ones for Python are

- TensorFlow, Keras
- PyTorch

Working with TensorFlow requires going into lot of details of the contruction of the computation graph, whereas Keras is a higher level interface for TensorFlow. Tensorflow is very popular in the industry and good for production code.

PyTorch can be used as low level interface, but is much more user-friendly than TensorFlow, but it also has a higher level interface. PyTorch is more popular in the research community.

In Stanford courses and projects, it is very common to use PyTorch.

## Main features that any deep learning library should provide

No matter what library or language you use, the main features provided by a deep learning library are
1. Use the GPU to speed up computation
2. Ability to do automatic differentiation
3. Useful library functions for common architectures and optimization algorithms

### PyTorch
We will look at all of the above in pytorch.
The best way to think about pytorch is that its numpy + GPU + autograd.

You can install it with

```conda install pytorch```.

Alternatively (and recommended), run this notebook in Google Colab-- it provides an environment with all of the PyTorch dependencies plus a GPU free of charge.

In [ ]:
import torch
import numpy as np

The equivalent object to numpy arrays in PyTorch are called tensors, but they are just multidimensional arrays.

In [ ]:
torch.tensor([2,3,4,5])

In [ ]:
torch.zeros((5,5))

In [ ]:
x = torch.ones((5,5))
print(type(x))
x

In [ ]:
2*x + 5

In [ ]:
torch.randn(5,5)

In [ ]:
x = torch.rand(25)
x

In [ ]:
x=x.reshape(-1,5)
x

In [ ]:
x.shape

In [ ]:
print(torch.arange(10))
print(torch.eye(5))
print(torch.linspace(0,1,10))

Some functions are a bit different

In [ ]:
A = np.random.rand(5,5)
x = np.ones((5,1))
print(A)
print(x)

A@x

In [ ]:
A_ = torch.rand((5,5))
x_ = torch.ones(5,1)
print(A_)
print(x_)
A_@x_

In [ ]:
# class Test:
#   def __init__(self):
#     self.__data = 'Great__'
#     self.data = 'Great'
# test = Test()
# print(test.__data)

In [ ]:
x = np.ones((5,1))
print(x)

In [ ]:
x_ = torch.ones(5,1)
print(x_)

You can convert tensors to a numpy array that shares its memory with the pytorch tensor -> to use more library that are compatible to numpy but not pytorch

In [ ]:
x = torch.ones(5,5)
x

In [ ]:
xn = x.numpy()
print(type(xn))
xn

In [ ]:
# Changes in Numpy will cause changes in Tensor
xn[4,2]=10
xn

In [ ]:
x

### Using the GPU

The GPU (Graphical Processing Unit) is a separate processing unit that is specialized to handle bulk computations required for rendering high quality graphics. It mainly consists of a large number of processor cores that are individually very slow, but because of their sheer number (around 2000) they can churn through computations very quickly.

In [ ]:
import torch
"""
CUDA is a parallel computing platform
and application programming interface
that allows software to use certain types
of graphics processing units for general purpose processing
""";
#CUDA -> Recommend CME213 (C++)
torch.cuda.is_available()

Installing the GPU drivers and the CUDA toolkit can be quite messy, so if you just want to experiment with GPUs and deep learning libraries, you can use [Google colaboratory](https://colab.research.google.com/)

In [ ]:
gpu = torch.device("cuda")
cpu = torch.device("cpu")

In [ ]:
A = torch.rand(100,100)
B = torch.rand(100,100)

In [ ]:
A@B

In [ ]:
A_gpu = A.to(gpu)
B_gpu = B.to(gpu)

In [ ]:
A_gpu@B_gpu

In [ ]:
A@B_gpu #this won't work!

In [ ]:
C_gpu = A_gpu@B_gpu
C = C_gpu.to(cpu)
C

## Speedup from GPU
`%%timeit` is a Jupyter Notebook magic command that is used to measure the execution time of a Python code snippet. When you add `%%timeit` at the beginning of a cell in a Jupyter Notebook, it will run the code in the cell multiple times and measure the average execution time.

In [ ]:
import time

In [ ]:
%%timeit
A = torch.rand(3000,3000)
B = torch.rand(3000,3000)
for i in range(5):
    B=torch.mm(A,B)


In [ ]:
%%timeit
A = torch.rand(3000,3000, device = gpu)
B = torch.rand(3000,3000, device = gpu)
for i in range(5):
    B=torch.mm(A,B)

## Automatic Differentiation

PyTorch uses dynamic computation graphs to compute the gradients of the parameters. Only certain parameters need gradients: i.e. the parameters we are wanting to update: weights and biases!

In [ ]:
x = torch.tensor([2.0])
w = torch.tensor([5.0], requires_grad = True)
b = torch.tensor([2.0], requires_grad = True)

In [ ]:
y = w*x + b #12 = 5*2 + 2
y

Define an error for your function

In [ ]:
loss = torch.norm( y - 13)
loss

Calling `x.backward()` on any tensor forces PyTorch to compute all the gradients of the tensors used to compute `x` which had the `requires_grad` flag set to `True`. The computed gradient will be stored in the `.grad` property of the tensors

In [ ]:
loss.backward()

In [ ]:
w.grad

In [ ]:
b.grad

We now can use the gradients of $w$ and $b$ to update their values (i.e. gradient descent). A helpful trick we can use here is telling PyTorch to skip the gradient calculations, which can help to reduce the memory usage and speed up computations. In this case, we don't want to update the gradients of $w$ and $b$ based on this parameter update.

In [ ]:
# It will reduce memory consumption for computations
# that would otherwise have requires_grad=True
with torch.no_grad(): #when we update w and b, do not update gradients
    w -= 0.01 * w.grad
    b -= 0.3 * b.grad

In [ ]:
w,b

In [ ]:
w.grad, b.grad

In [ ]:
# the gradients will accumulate and lead to incorrect updates and slower convergence.
w.grad.zero_()
b.grad.zero_()

w.grad, b.grad

After our update to our parameters, we can do another run through the network (forward propagation).

In [ ]:
y = w*x + b

In [ ]:
y

As we can see, our prediction for $y$ has improved from 12 -> 12.34, meaning our $w$ and $b$ approximations have improved. We can calculate the loss again to see that it has improved, and again use backpropagation to update the gradients of $w$ and $b$.

In [ ]:
loss = torch.norm( y - 13)
loss

In [ ]:
loss.backward()
w.grad, b.grad

### Making it more compact

In [ ]:
def model_fn(x,w,b):
    return w*x + b

In [ ]:
def loss_fn(y,yt):
    return torch.norm(y-yt)

In [ ]:
w = torch.tensor([5.0], requires_grad = True)
b = torch.tensor([2.0], requires_grad = True)

In [ ]:
x = torch.tensor([2.0])
yt = torch.tensor([13.0])

In [ ]:
y = model_fn(x,w,b)
loss = loss_fn(y,yt)
loss.backward()
with torch.no_grad():
    w -= 0.05 * w.grad
    b -= 0.05 * b.grad
w.grad.zero_()
b.grad.zero_()

print( f" w = {w}\n b = {b}\n y = {y}\n loss = {loss}")
#note that 'loss' indicates the loss for the previous values
for epoch in range(1,5):
    # running again
    print(15*'-')
    y = model_fn(x,w,b)
    loss = loss_fn(y,yt)
    loss.backward()
    with torch.no_grad():
        w -= 0.05 * w.grad
        b -= 0.05 * b.grad
    w.grad.zero_()
    b.grad.zero_()

    print( f"epoch = {epoch}\n w = {w} \n b = {b}\n y = {y}\n loss = {loss}")


### Slightly more complicated problem

1. Forward propagation to get predicted yhat
2. Get the loss by loss_fun(y, yhat)
3. Backward propagation to get the gradient
4. Update parameters
5. Repeat 1 ~ 4 until convergence

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def model_fn(x,w,b):
    return w@x + b

In [ ]:
def loss_fn(y,yt):
    return torch.norm(y-yt)

In [ ]:
w = torch.rand((5,5), requires_grad = True)
b = torch.ones((5,1), requires_grad = True)

In [ ]:
x = torch.randn(5,100)
yt = torch.randn(1,100)
losses = []

In [ ]:
for i in range(50):
  # 1. Forward
  y = model_fn(x,w,b)
  # 2. Get loss
  loss = loss_fn(y,yt)
  # 3. backward
  loss.backward()
  # 4. Update
  with torch.no_grad():
      w -= 0.05 * w.grad
      b -= 0.05 * b.grad
  w.grad.zero_()
  b.grad.zero_()

  losses+=[loss.item()]
  print( f"loss = {loss}")
  plt.plot(losses);

## Using Library functions

In [ ]:
model = torch.nn.Sequential(
    # create layer of sequence
    # F(WX + B)
    # First layer: ReLU(WX + B)
      # ReLU is important because it is doing some non-linear tranformation
    torch.nn.Linear(5, 5),
    torch.nn.ReLU(),

    # Output layer
    torch.nn.Linear(5, 1),
)

In [ ]:
list(model.parameters())

In [ ]:
#mean square loss → MSE = (1/n) * sum((y_pred - y_true)^2)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
x = torch.randn(100,5)
yt = torch.randn(100,1)
losses = []

Using the optim package to get an optimizer (we hard-coded standard gradient descent to update $w$ and $b$, here it will update itself based on whatever optimizer we choose).

Common optimizers:

*   Adam
*   SGD (Stochastic Gradient Descent)



In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)

`optimizer.step()` is used to update the model parameters based on the gradients computed during backpropagation. During training, the optimizer computes the gradients of the loss function with respect to the model parameters
  
`optimizer.zero_grad()` is used to set the gradients of all the model parameters to zero before computing the gradients for the next batch of data. If we don't zero out the gradients before computing the gradients for the next batch, the gradients will accumulate, leading to incorrect updates and slower convergence.

In [ ]:
for i in range(1000):
    # feed data through model to get prediction
    y = model(x)

    # calculate loss (compare prediction to true value)
    loss = loss_fn(y,yt)

    # calculate gradients of parameters
    loss.backward()

    #make the parameter to take a step -> update the parameter
    optimizer.step()

    # clear gradients
    optimizer.zero_grad()

    losses+=[loss.item()]
    print( f"loss = {loss}")
plt.plot(losses);

## MNIST Example

In [ ]:
from torchvision.datasets import MNIST

In [ ]:
data = MNIST(".",download=True)

In [ ]:
len(data)

In [ ]:
import numpy as np
img,y = data[np.random.randint(1,60000)]
print(y)
img

In [ ]:
img,y = data[2]
img

In [ ]:
print(data.data[2].shape)

In [ ]:
print(data.targets[2])

### MNIST Training

In [ ]:
model = torch.nn.Sequential( # 28*28 = 784
    torch.nn.Linear(784,  100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 10),
    torch.nn.LogSoftmax(dim=1)
)

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
sample = np.random.choice(range(len(data.data)),1000)
x = data.data[sample].reshape(1000,-1).float()/255
yt = data.targets[sample]

In [ ]:
x.shape,yt.shape

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)
losses = []

In [ ]:
for i in range(100):
    # get data and true labels
    sample = np.random.choice(range(len(data.data)),1000)
    x = data.data[sample].reshape(1000,-1).float()/255
    yt = data.targets[sample]

    # get prediction
    y = model(x)

    # input (Tensor) – Predicted unnormalized logits
    # target (Tensor) – Ground truth class indices or class probabilities
    loss = loss_fn(y,yt)

    # compute gradients
    loss.backward()

    # update parameters and clear previous gradients
    optimizer.step()
    optimizer.zero_grad()

    losses+=[loss.item()]
    #print( f"loss = {loss}")
plt.plot(losses);

In [ ]:
x_test = data.data[-1000:].reshape(1000,-1).float()/255
y_test = data.targets[-1000:]

In [ ]:
with torch.no_grad():
    y_pred = model(x_test)

In [ ]:
y_test[4]

In [ ]:
y_pred.argmax(dim=1)[4]

In [ ]:
print("Accuracy = ", (y_pred.argmax(dim=1) == y_test).sum().float().item()/1000.0)

## Course Conclusion

1. By now you should have a sufficient introduction to the various ways one can use python for scientific computing. The best way to learn more is to start using python for whatever project you are working on. Only practice will make you comfortable with using python.   

Recommended Project Source: kaggle  

Recommended ML/DL Courses: CS229, 230, 231N, 224 series, 238, 246  

2. I appreciate your time to submit the course feedback, which means a lot to me and improvement for this course in the future  

